# Introducción a TensorFlow Extended (TFX)
TensorFlow Extended (TFX) es una plataforma de machine learning de escala productiva provista por Google basada en TensorFlow. Provee de un marco de configuración y bibliotecas compartidas para integrar componentes comunes para definir, ejecutar y monitorear un sistema de machine learning.

Componentes opcionales:
- TensorFlow Serving: Sistema de otorgamiento de modelos de machine learning diseñado para entornos de producción. Facilita desplegar nuevos algoritmos y experimentos, manteniendo la misma arquitectura de servidor y APIs.
![TFX Serving Architecture](images/tfx-serving-architecture.png)
- TensorFlow JS: Librería de machine learning en JavaScript para entrenar y desplegar modelos de machine learning en el browser y con Node.js. Encapsula abstracciones, incluyendo objetos como Tensors y Optimizers. ml5.js es una librería que provee de acceso a algoritmos y modelos de machine learning con un API conciso y fácil de usar. Por medio de TensorFlow.js es posible: programar redes neuronales en JavaScript, convertir modelos pre entrenados en Python a TensorFlow.js, visualizar el comportamiento de modelos TensorFlow.js, preparar datos para procesamiento con TensorFlow.js.
- TensorFlow Lite: Marco de trabajo para deep learning de código abierto orientado a inferencia en dispositivos. Provee de un conjunto de herramientas para que los desarrolladores ejecuten modelos TensorFlow en dispositivos móvil, empotrados e IoT. Permite inferencia machine learning en los dispositivos para mejorar el tamaño de los binarios y el desempeño. Consta de un intérpete que corre modelos especialmente optimizados en diversos tipos de hardware y un convertidor que transforma modelos TensorFlow para ser empleados por el intérprete y que puede introducir optimizaciones para mejorar el tamaño del binario y el desempeño.

# Requisitos
TFX requiere de TensorFlow versión 2.1, por lo que es necesario crear un ambiente para tal versión como está descrito en la sección de anexos.



## Instalación de TFX en un ambiente TensorFlow < 2.1
Al instalar TFX via pip

`!pip install tensorflow`

`!pip install tfx`

en un ambiente TensorFlow 2.0 se generan los siguientes errores:

```
ERROR: tensorflow-serving-api 2.1.0 has requirement tensorflow~=2.1.0, but you'll have tensorflow 2.0.0 which is incompatible.
ERROR: tensorflow-data-validation 0.21.2 has requirement avro-python3!=1.9.2.*,!=<2.0.0,>=1.8.1; python_version >= "3.0", but you'll have avro-python3 1.9.2.1 which is incompatible.
ERROR: ml-metadata 0.21.1 has requirement tensorflow!=2.0.*,<3,>=1.15, but you'll have tensorflow 2.0.0 which is incompatible.
ERROR: Cannot uninstall 'PyYAML'. It is a distutils installed project and thus we cannot accurately determine which files belong to it which would lead to only a partial uninstall.
```    

# Panorama general de los componentes de TFX
Diagrama de flujo de datos de los componentes de TFX
![TFX Components](images/tfx-components.png)
## TFX Pipeline
Define un flujo de datos a través de múltiples componentes, con la meta de implementar una tarea específica de machine learning (p.e. constuir y desplegar un modelo de regresión con datos específicos). Los componentes Pipeline se construyen con librerías TFX. El resultado de un pipeline es un objetivo de despliegue TFX o un servicio de una petición de inferencia.
## Artifactos
En un pipeline, un artifacto es una unidad de datos que se comunica entre componentes. Generalmente los componentes tienen al menos un artifacto de entrada y un artifacto de salida. Todos los artifactos requieren tener metadatos asociados que definen el tipo y propiedades del artifacto. Los artifactos tienen que tener un tipo de artifacto definido en el repositorio metadatos de Machine Learning. 

Ejemplos de tipos de artifactos:
- Example: representa ejemplos materializados en TFRecord del buffer de protocolo `tensorflow::Example`, CSV, JSON o cualquier otro formato físico. Sin importar el formato físico, la forma en la que los Examples son empleados en un pipeline es exactamente la misma: se analizan para generar estadísticas, se validan contra un esquema esperado, se preprocesan en preparación para entrenamiento, se proveen a un Trainer para entrenar modelos, entre otras.
- Model: Puede representar un modelo entrenado exportado a diversos formatos físicos tales como TensorFlow SavedModel, ONNX, PMML o PKL (varios tipos tipos objetos Python que representan modelos). En todo caso, los modelos siempre se evalúan, analizan y despliegan para servir en pipelines.

El tipo de artifacto tiene un nombre único y una esquema de propiedades de sus instancias. TFX utiliza el tipo de artifacto como para determinar el uso del artifacto por componentes en el pipeline, no necesariamente para determinar el contenido físico del artifacto en el sistema de archivos.

In [3]:
import os
import pprint
import tempfile
import urllib

import absl
import tensorflow as tf
tf.get_logger().propagate = False
pp = pprint.PrettyPrinter()

import tfx


In [5]:
from tfx.components import CsvExampleGen
from tfx.components import Evaluator
from tfx.components import ExampleValidator
from tfx.components import ModelValidator
from tfx.components import Pusher
from tfx.components import SchemaGen
from tfx.components import StatisticsGen
from tfx.components import Trainer
from tfx.components import Transform
from tfx.orchestration import metadata
from tfx.orchestration import pipeline
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
from tfx.proto import evaluator_pb2
from tfx.proto import pusher_pb2
from tfx.proto import trainer_pb2
from tfx.proto.evaluator_pb2 import SingleSlicingSpec
from tfx.utils.dsl_utils import external_input

%load_ext tfx.orchestration.experimental.interactive.notebook_extensions.skip

ImportError: cannot import name 'record_based_tfxio' from 'tfx_bsl.tfxio' (C:\Users\Marciano\.conda\envs\tf21\lib\site-packages\tfx_bsl\tfxio\__init__.py)

In [ ]:
Verificar las versiones de las bibliotecas.

In [4]:
print('TensorFlow version: {}'.format(tf.__version__))
print('TFX version: {}'.format(tfx.__version__))

TensorFlow version: 2.1.0
TFX version: 0.21.0


Configurar las rutas del pipeline.

Descargar datos de ejemplo

In [6]:
# This is the root directory for your TFX pip package installation.
_tfx_root = tfx.__path__[0]

# This is the directory containing the TFX Chicago Taxi Pipeline example.
_taxi_root = os.path.join(_tfx_root, 'examples/chicago_taxi_pipeline')

# This is the path where your model will be pushed for serving.
_serving_model_dir = os.path.join(
    tempfile.mkdtemp(), 'serving_model/taxi_simple')

# Set up logging.
absl.logging.set_verbosity(absl.logging.INFO)

In [7]:
_data_root = tempfile.mkdtemp(prefix='tfx-data')
DATA_PATH = 'https://raw.githubusercontent.com/tensorflow/tfx/master/tfx/examples/chicago_taxi_pipeline/data/simple/data.csv'
_data_filepath = os.path.join(_data_root, "data.csv")
urllib.request.urlretrieve(DATA_PATH, _data_filepath)

('C:\\Users\\Marciano\\AppData\\Local\\Temp\\tfx-datam08aga56\\data.csv',
 <http.client.HTTPMessage at 0x203198c15c8>)

In [8]:
head {_data_filepath}

SyntaxError: invalid syntax (<ipython-input-8-3c6c1d7d67a5>, line 1)

In [9]:
# Here, we create an InteractiveContext using default parameters. This will
# use a temporary directory with an ephemeral ML Metadata database instance.
# To use your own pipeline root or database, the optional properties
# `pipeline_root` and `metadata_connection_config` may be passed to
# InteractiveContext. Calls to InteractiveContext are no-ops outside of the
# notebook.
context = InteractiveContext()

NameError: name 'InteractiveContext' is not defined